In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *
import torch
import torch.nn as nn
from nlputils import split_wiki,get_wiki

In [2]:
# bs=48
# bs=24
bs=64
lang = 'vi'

In [3]:
# torch.cuda.set_device(1)

In [4]:
data_path = Config.data_path()

In [5]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']

In [6]:
get_wiki(path,lang)

/home/tuna/.fastai/data/viwiki/viwiki already exists; not downloading


In [7]:
dest = split_wiki(path,lang)

/home/tuna/.fastai/data/viwiki/docs already exists; not splitting


In [8]:
dest.ls()[:5]

[PosixPath('/home/tuna/.fastai/data/viwiki/docs/WrestleMania 36.txt'),
 PosixPath('/home/tuna/.fastai/data/viwiki/docs/Yến Lan.txt'),
 PosixPath('/home/tuna/.fastai/data/viwiki/docs/The Hardest Part (bài hát).txt'),
 PosixPath('/home/tuna/.fastai/data/viwiki/docs/Trận Maastricht.txt'),
 PosixPath('/home/tuna/.fastai/data/viwiki/docs/Olave Baden-Powell.txt')]

In [9]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=4))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(60000, 68790)

In [10]:
# data = load_data(path, f'{lang}_databunch', bs=bs)

In [11]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False).to_fp16()

In [12]:
learn.model = nn.DataParallel(learn.model)

In [13]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [14]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.605741,3.599135,0.359009,1:32:56
1,3.580776,3.655813,0.351114,1:32:46
2,3.669741,3.658460,0.350824,1:33:07
3,3.600712,3.619071,0.355536,1:32:50
4,3.580138,3.572835,0.359893,1:33:05
5,3.595680,3.512127,0.366577,1:33:10
6,3.511461,3.422906,0.376792,1:33:07
7,3.369179,3.335427,0.387454,1:33:08
8,3.383684,3.254117,0.397860,1:33:07
9,3.333484,3.218723,0.402909,1:33:07


/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1269: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


In [33]:
# mdl_path = '/models'
# # mdl_path.mkdir(exist_ok=True)
# learn.to_fp32().save('/home/tuna/FDM/MarketSentiment/ULMFit-sentiment/', with_opt=False)
# learn.data.vocab.save('vocab' + '.pkl')

In [40]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [16]:
learn.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.336854,3.218720,0.402911,2:43:06
1,3.321256,3.218720,0.402911,2:43:36


In [21]:
lm_fns

['vi_wt', 'vi_wt_vocab']

In [29]:
learn.path/learn.model_dir

PosixPath('/home/tuna/.fastai/data/viwiki/docs/models')

PosixPath('/home/tuna/.fastai/data')